# **Modélisation et arbitrage de la surface de volatilité implicite (options vanilles)**

En tant que trader exotiques, tu reçois une demande sur Bloomberg IB :


J’ai un Down-and-Out Call sur AAPL pour toi.

Strike : 220

Barrière : 190

Échéance : 17 janvier 2026

Rebate : 0 (donc si la barrière est touchée → option s’annule sans compensation)

Taille : 100 000 actions (notional)

Donne-moi ton meilleur prix bid/ask (deux sens) pendant quelques minutes.

 ## Black-Scholes Model

Solving the Black-Scholes Equation Yields:

$$C = S_t \Phi(d_1) - Ke^{-rt} \Phi(d_2)$$

$$\Phi(x) = \int_{-\infty}^x \frac{1}{\sqrt{2\pi}}e^{\frac{-s^2}{2}}ds$$

$$d_1 = \frac{ln(\frac{S_t}{K})+(r+\frac{\sigma^2}{2})t}{\sigma \sqrt{t}}$$

$$d_2 = d_1 - \sigma \sqrt{t}$$


# Formule de Black–Scholes

Mathématiquement, la formule de Black–Scholes est un **fonctionnel**, c’est-à-dire une application :  

$$ \mathbb{R}^5 \longrightarrow \mathbb{R} $$

$$ BS : (S, K, r, \sigma, T) \;\mapsto\; C $$

Où :  
- $S \in \mathbb{R}^+$ : prix du sous-jacent (*spot price*)  
- $K \in \mathbb{R}^+$ : prix d’exercice (*strike price*)  
- $r \in \mathbb{R}$ : taux sans risque (*risk-free rate*)  
- $\sigma \in \mathbb{R}^+$ : volatilité (*volatility*)  
- $T \in \mathbb{R}^+$ : temps restant avant l’échéance (*time to expiry*)  

Ainsi :  
$$ C \in \mathbb{R}^+ \quad \text{correspond au prix de l’option call européenne.} $$


In [ ]:
import numpy as np
from scipy.stats import norm

def black_scholes_call(S, K, r, sigma, T):
    """
    Calculate Black-Scholes price for a European call option.

    Parameters:
    S (float): Current stock price
    K (float): Strike price
    r (float): Risk-free interest rate
    sigma (float): Volatility
    T (float): Time to expiration (in years)

    Returns:
    float: Call option price
    """
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    call = S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    return call


Cela peut sembler étrange : nous avons déjà toutes les données nécessaires pour utiliser le modèle de Black-Scholes (sauf la volatilité).

En réalité, nous connaissons aussi le prix de l’option – car celui-ci est fixé par l’offre et la demande sur le marché !





In [ ]:
S = 100
K = 100
r = 0.05
T = 1

# Grâce à l'offre et la demande, le prix de marché de l'option d'achat est fourni par le marché/contrat
C = 9.36 # Le prix de l'option call est donné par le marché/contrat

# La volatilité n'est PAS observable ! On peut l'estimer avec différents indicateurs mais elle n'est pas directement disponible.
sigma = 0.2

print("Le prix Black-Scholes de l'option call est : $", np.round(black_scholes_call(S, K, r, sigma, T), 2))


On met en place un **problème d’optimisation** en utilisant le calcul différentiel classique :

$$
\sigma^* = \min_{\sigma} \; (C_{mkt} - C_{BS})^2
$$

où :

- $C_{mkt}$ = le **prix observé sur le marché** de l’option (issu de l’offre et la demande).  
- $C_{BS}$ = le **prix théorique** donné par la formule de Black-Scholes pour une volatilité $\sigma$.  
- $\sigma^*$ = la **volatilité implicite** Black–Scholes.  


In [ ]:

from scipy.optimize import newton
def implied_volatility(C,S, K, r, T,sigma_init=0.2):
    """
    Calcule la volatilité implicite en utlisant la methode de Newton
    paramètres:
    sigma_init:Estimation initiale de la volatilité

    """
    def objective(sigma):
        # Difference au carré entre le prix théorique et le prix du marché
        return (black_scholes_call(S, K, r, sigma, T) - C)**2
    #utilisation de la methode de newton pour resoudre objective(sigma)=0
    implied_vol= newton(objective,sigma_init)
    return implied_vol

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

def plot_implied_vol_convergence(C, S, K, r, T, sigma_init=0.2):
    """Trace les étapes de convergence de la méthode de Newton-Raphson pour la volatilité implicite"""

    # Fonction objectif : différence au carré entre le prix de l'option selon Black-Scholes et le prix observé
    def objective(sigma):
        return (black_scholes_call(S, K, r, sigma, T) - C)**2

    # Approximation de la dérivée de la fonction objectif (différence finie)
    def derivative(sigma):
        h = 1e-7
        return (objective(sigma + h) - objective(sigma)) / h

    # Listes pour suivre la convergence
    sigmas = []
    errors = []
    sigma = sigma_init

    # Boucle de Newton-Raphson
    for i in range(10):
        sigmas.append(sigma)
        error = abs(objective(sigma))
        errors.append(error)

        # Arrêt si l'erreur est suffisamment petite
        if error < 1e-10:
            break

        # Mise à jour de sigma selon la méthode de Newton-Raphson
        sigma = sigma - objective(sigma)/derivative(sigma)


    plt.figure(figsize=(8, 6))
    plt.semilogy(range(len(errors)), errors, 'bo-')  # échelle logarithmique pour l'erreur
    plt.grid(True, alpha=0.3)
    plt.xlabel('Itération')
    plt.ylabel('Erreur absolue ($)')
    plt.title('Convergence de Newton-Raphson pour la volatilité implicite')

    plt.show()

# Tracé de la convergence pour l'exemple donné
plot_implied_vol_convergence(C=12.15, S=100, K=100, r=0.05, T=1)








In [ ]:

S = 100
K = 100
r = 0.05
T = 1
C = 10.15  # Prix de marché de l’option call

# Calcul de la volatilité implicite pour cet exemple
implied_vol = implied_volatility(C, S, K, r, T)
print(f"La volatilité implicite est : {implied_vol:.1%}")

# Vérification du résultat : on recalcule le prix du call avec la volatilité trouvée
price_check = black_scholes_call(S, K, r, implied_vol, T)
print(f"Prix de marché observé : ${C:.2f}")
print(f"Prix obtenu avec la volatilité implicite : ${price_check:.2f}")


### Remarques :
Par souci de cohérence, lorsqu’on parle de volatilité implicite, on fait généralement référence à la volatilité implicite de Black-Scholes, c’est-à-dire l’inverse de la fonction de valorisation Black-Scholes.
En d’autres termes, au lieu de partir d’une volatilité pour calculer le prix de l’option, on part du prix de marché observé pour en déduire la volatilité correspondante.

Dans certains contextes, si ce terme peut prêter à confusion, il sera précisé qu’il s’agit d’une volatilité implicite calculée à partir d’un autre modèle (par exemple Heston ou SABR).

La volatilité implicite utilise donc les prix de marché actuels comme référence afin de déterminer le paramètre de volatilité du modèle de Black-Scholes. Cela nous donne une information précieuse sur le niveau de volatilité anticipé par le marché, c’est-à-dire une mesure de l’incertitude ou du risque que les traders intègrent dans la valorisation des contrats d’options.

###  **La surface de volatilité implicite**

>« Les prix de marché nous disent ce que les traders pensent, mais la volatilité implicite nous dit ce qu’ils craignent. »

La volatilité implicite est simplement la volatilité nécessaire pour reproduire le prix d’équilibre d’une option dans le cadre du modèle de Black-Scholes, étant donné que le marché et le contrat fixent les autres paramètres.

Cela nous permet de visualiser un concept essentiel : la surface de volatilité implicite.


En pratique, la surface de volatilité implicite est construite en utilisant :

Les options Put hors de la monnaie (OTM Puts)

Les options à la monnaie (ATM Calls/Puts) qui, par la parité Put/Call, devraient afficher la même volatilité implicite

Les options Call hors de la monnaie (OTM Calls)

Voici à quoi peut ressembler une surface de volatilité implicite sur actions :

Les options de protection à la baisse (puts OTM) sont généralement plus chères que les calls OTM → cela reflète une peur plus forte du marché sur la baisse.

À l’inverse, sur certaines classes d’actifs comme les matières premières, c’est souvent la hausse qui fait peur → les options d’achat (calls OTM) deviennent plus coûteuses que les puts.

In [ ]:
import plotly.graph_objects as go

# Données pour la surface de volatilité

strikes=[90,95,100,105,110]# Prix d'exercice (strikes)

maturities=[1/12,3/12,6/12,1,2]
maturity_labels=['1 Mois', '3 Mois', '6 Mois', '1 An', '2 Ans']

# Chaque ligne correspond à une maturité et chaque colonne à un strike
vols = np.array([
    [28.0, 24.5, 22.0, 20.5, 19.5],  # Pour 1 mois
    [27.5, 24.0, 21.8, 20.3, 19.3],  # Pour 3 mois
    [27.0, 23.5, 21.5, 20.0, 19.0],  # Pour 6 mois
    [26.5, 23.0, 21.2, 19.8, 18.8],  # Pour 1 an
    [26.0, 22.5, 21.0, 19.5, 18.5]   # Pour 2 ans
])

# cela permet d'associer chaque strike a chaque maturité
X, Y= np.meshgrid(strikes, maturities)

# Création de la surface 3D

fig=go.Figure(data=[go.Surface(
    x=X,  # axe des stikes
    y=Y,   # axe des maturités
    z=vols,  # Axe des volatilités implicites
    colorscale='Viridis', # Palette de couleurs
)])

# Mise en forme et personalisation

fig.update_layout(
    title='Surface de volatilité implicite',
    scene=dict(
        xaxis_title='Strike',
        yaxis_title='Maturité (Années)',
        zaxis_title='Volatilité implicite (%)',
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.2)          # Angle de vue de la caméra
        )
    ),

    width=800,
    height=600,
    template = 'plotly_dark', #Thème sombre
    paper_bgcolor='rgb(30,30,30)', #couleur de fond exterieur
    plot_bgcolor='rgb(30,30,30)'


)

fig.show()




In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig=go.Figure()
fig2= go.Figure()
#========Tracé du skew : volatilité en fonction du strike pour chaque maturité

for i , (maturity, label) in enumerate(zip(maturities, maturity_labels)):
    fig.add_trace(
        go.Scatter(
            x=strikes,
            y=vols[i],
            name=label,
            mode='lines+markers'

        )
    )

# Tracé de la Terme structure : Volatilié en fonction de la maturité pour chaque strike

for i, strike in enumerate(strikes):
    fig2.add_trace(
        go.Scatter(
            y=vols[:,i],
            x=maturities,
            name=f'Strike {strike}',
            mode='lines+markers'
        )
    )


# Mise en page du graphique Skew

fig.update_layout(
    title='Skew (Volatilité en fonction du strike pour chaque maturité)',
    xaxis_title='Strike',
    yaxis_title='Volatilité implicite (%)',
    legend_title='Maturité (Années)',
    width=500,
    height=400,
    showlegend=True
)

# Mise en page du graphique Term Structure
fig2.update_layout(
    title='Term Structure (Volatilié en fonction de la maturité pour chaque strike)',
    xaxis_title='Maturité (Années)',
    yaxis_title='Volatilité implicite (%)',
    width=500,
    height=400,
    showlegend=True
)

# Combiner les deux graphiques Cote a cote

combined_fig = make_subplots(rows=1, cols=2, subplot_titles=('Skew', 'Term Structure'))

# Ajouter les courbes du skew dans la première colonne
for trace in fig.data:
    combined_fig.add_trace(trace, row=1, col=1)

for trace in fig2.data:
    combined_fig.add_trace(trace, row=1, col=2)

# Mise en page du graphique combiné
combined_fig.update_layout(
    template='plotly_dark',
    width=1000,
    height=400,
    paper_bgcolor='rgb(30,30,30)',
    plot_bgcolor='rgb(30,30,30)',
    showlegend=True
)

combined_fig.show()





### **Le Problème (Volatilités Incohérentes, Arbitrage)**

Le problème est le suivant :
BS:(S,K,r,σ,T)↦C

Même si la volatilité implicite varie selon les paramètres de marché et du contrat, dans le modèle de Black-Scholes $\sigma$ est supposée constante !

Mais dans la réalité : quelle volatilité doit-on choisir sur la surface, si elle diffère pour chaque combinaison marché/contrat ?

Pour valoriser correctement les options dans une salle de marché, il faut être cohérent sur toutes les options, sinon on s’expose à des opportunités d’arbitrage !




### Exemple : Butterfly Spread mal valorisé (Arbitrage sur la volatilité)

Même si ce n’est pas un vrai butterfly spread :

Imagine que nous faisons du Market Making avec un modèle Black-Scholes à volatilité constante (σ = 22,5%) pour une action qui se négocie à 100 $.

Nous proposons des marchés acheteur/vendeur pour :

Call A (Strike 90) :

Notre prix (avec σ = 22,5%) : 12,50 $

Prix du marché (volatilité implicite 25%) : 13,20 $

Call B (Strike 110) :

Notre prix (σ = 22,5%) : 4,20 $

Prix du marché (volatilité implicite 20%) : 3,80 $

Opportunité pour un trader malin :

Acheter Call A à notre prix (nous le vendons trop bon marché)

Vendre Call B à notre prix (nous le vendons trop cher)

Faire un delta hedge pour neutraliser le risque directionnel

In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

S0=100
K1, K2= 90,100
r=0.02
T=1.0
n_paths=10000 # Nombre de chemin simulés
n_steps=252  # Pas de temps(journaliers d'un an)

# Volatilités du marché vs notre volatilité constante

sigma_market_K1=0.25
sigma_market_K2=0.2
sigma_our=0.225 # Notre volatilité constante

# Fonction prix d'un call BS
def bs_call(S,K,r,sigma,T):
  d1=(np.log(S/K)+(r+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
  d2=d1-sigma*np.sqrt(T)
  return S*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2)

# Calcul des prix initiaux

our_price_K1=bs_call(S0,K1,r,sigma_our,T)
our_price_K2=bs_call(S0,K2,r,sigma_our,T)
market_price_K1=bs_call(S0,K1,r,sigma_market_K1,T)
market_price_K2=bs_call(S0,K2,r,sigma_market_K2,T)

# Simulation des chemins de prix de l'action
dt = T/n_steps
W=np.random.standard_normal((n_steps+1,n_paths)) # Chemins de Wiener
W[:,0]=0
# Pour simplification , on utilise une volatilité moyenne pour la simulation
sigma_realized=(sigma_market_K1+sigma_market_K2)/2
S=np.zeros((n_paths, n_steps+1))
S[:,0]=S0
# Généraation des  chemins de prix
for t in range(1,n_steps+1):
  S[:,t]=S[:,t-1]*np.exp((r-0.5*sigma_realized**2)*dt+sigma_realized*np.sqrt(dt)*W[t])


#=========Calcul des P&L pour chque chemin=======
desk_pnl=np.zeros(n_paths) # PnL du desk(nous le market maker)
trader_pnl=np.zeros(n_paths) # PnL du trader(la contrepartie)

for i in range(n_paths):
    # Desk vend K1 trop bon marché et achète K2 trop cher
    desk_pnl[i]=-(max(S[i,-1] -K1,0)-our_price_K1) # Perte sur K1
    desk_pnl[i]+=(max(S[i,-1]-K2,0)-our_price_K2) # Perte sur K2

    # Trader prend la position opposée
    trader_pnl[i]=-desk_pnl[i]


# Visualisation de la distribution des P&L

plt.figure(figsize=(12, 6))
plt.hist(desk_pnl, bins=50, alpha=0.5, label='P&L Desk', color='red')
plt.hist(trader_pnl, bins=50, alpha=0.5, label='P&L Trader', color='green')
plt.axvline(x=np.mean(desk_pnl), color='red', linestyle='--',
            label=f'P&L moyen Desk: ${np.mean(desk_pnl):.2f}')
plt.axvline(x=np.mean(trader_pnl), color='green', linestyle='--',
            label=f'P&L moyen Trader: ${np.mean(trader_pnl):.2f}')
plt.title('Distribution des P&L - Arbitrage sur la surface de volatilité')
plt.xlabel('P&L ($)')
plt.ylabel('Fréquence')
plt.legend()
plt.grid(True)
plt.show()


print(f"Probabilité de perte du Desk: {np.mean(desk_pnl < 0):.1%}")
print(f"Probabilité de perte du Trader: {np.mean(trader_pnl < 0):.1%}")
print(f"Espérance de P&L du Desk: ${np.mean(desk_pnl):.2f}")
print(f"Espérance de P&L du Trader: ${np.mean(trader_pnl):.2f}")






Le desk (nous) valorise les options avec σ = 22,5%.

Le marché réel a des volatilités différentes selon le strike :

K1 = 90 → σ marché = 25%

K2 = 110 → σ marché = 20%

Donc nos prix sont incohérents par rapport au marché.

Call K1 → notre prix trop bas → on vend moins cher que le marché → on perd si quelqu’un achète.

Call K2 → notre prix trop haut → on vend plus cher que le marché → on perd si on achète nous-mêmes.

In [ ]:
# Parametres de la simulation

n_trades=1000 # Nombre total de trades simulés
n_paths=100 # Nombre de trajectoires simulées
initial_wealth=1000 # Richesse initiale (capitale de depart pour le desk et le trader )

# Probabilités et esperance issues de la simulation precedente

p_desk_loss=0.504    # Proba que le desk perde sur un trade
desk_ev=-0.25         # Esperance de gain par trade pour le desk
p_trader_loss=0.496  # Proba que le trader perde
trader_ev=0.25 # Esperance de gain par trde pour le trader


# Initialisation des trajectoires

desk_wealth=np.zeros((n_paths,n_trades+1)) # Richesse du desk au fil des trades
trader_wealth=np.zeros((n_paths,n_trades+1)) # Richesse du trader au fil des trades
desk_wealth[:,0]=initial_wealth
trader_wealth[:,0]=initial_wealth

# Simulation des trades
for path in range(n_paths): # Pour chaque trajectoire simulée

    for trade in range(n_trades): # Pour chaque trade de la trajectoire
        # On tire un résultat aleatoire (perte/gain) en respectant les proba
        desk_outcome = np.random.choice([-1,1], p=[p_desk_loss,1-p_desk_loss])

        #On ajuste la taille du gain/perte pour que l'esperance soit bien =desk_ev
        if desk_outcome==1:
            desk_gain=abs(desk_ev/p_desk_loss) #gain si desk gagne
        else:
            desk_gain=-abs(desk_ev/(1-p_desk_loss)) # Perte si desk perde

        #Mise a jour de la richesse du desk et du trader
        desk_wealth[path,trade+1]=desk_wealth[path,trade]+desk_gain
        trader_wealth[path,trade+1]=trader_wealth[path,trade]-desk_gain

# Visualisation des resultats

plt.figure(figsize=(12,6))
trade_nums=np.arange(n_trades+1) # Numéros des trades

for path in range(n_paths):
    plt.plot(trade_nums,desk_wealth[path,:],color='red',alpha=0.1) # Desk en rouge
    plt.plot(trade_nums,trader_wealth[path,:],color='green',alpha=0.1) # Trader en vert


# Tracer les moyennes des trajectoires
plt.plot(trade_nums,np.mean(desk_wealth,axis=0),color='red',label='Moyenne du Desk',linewidth=2)
plt.plot(trade_nums,np.mean(trader_wealth,axis=0),color='green',label='Moyenne du Trader',linewidth=2)


plt.title('Évolution de la richesse au fil des trades')
plt.xlabel('Nombre de trades')
plt.ylabel('Richesse ($)')
plt.legend()
plt.grid(True)
plt.show()

print(f"Richesse finale moyenne du Desk : ${np.mean(desk_wealth[:, -1]):.2f}")
print(f"Richesse finale moyenne du Trader : ${np.mean(trader_wealth[:, -1]):.2f}")

Ainsi, nous ne pouvons pas utiliser le modèle de Black-Scholes pour fournir des cotations – il n’est même pas cohérent pour les options vanilles dans le cadre de son propre modèle.

Si nous ne sommes pas disposés à l’utiliser pour évaluer des options vanilles, comment pourrions-nous l’appliquer à quelque chose de plus illiquide ou à un produit exotique OTC ?

### Les solutions:

Heureusement, de nombreuses personnes très brillantes ont proposé des solutions à ce problème, en développant des modèles alternatifs permettant d’ajuster un ensemble de paramètres à une surface de volatilité afin de garantir des prix cohérents !

Chacun présente des avantages et des inconvénients – certains assurent l’absence d’arbitrage dans les prix, d’autres capturent des heuristiques de marché plus riches et des faits stylisés, et d’autres encore privilégient l’efficacité au détriment de ces aspects.

Modèles alternatifs

Volatilité locale (par exemple, Dupire/Derman)

Volatilité stochastique (par exemple, Heston/SABR)

Sauts de diffusion (par exemple, Merton)

Modèles rugueux (Rough Models, par exemple, le modèle Rough Bergomi de Gatheral)